In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install torch
!pip install transformers==4.30.2
!pip install pandas==2.0.3
!pip install numpy==1.24.3
!pip3 install Cython

In [ ]:
# Import models and data
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import Adam
import torch.nn.functional as F
import data_load

주석 처리된 '!git clone https://github.com/hyunmin5366/NLPClassification.git' 부분은 깃허브에서 미니 데이터 셋을 불러오는 부분인데, 전체 데이터 셋은 아래 설명을 따라 다운로드 해주세요.

### 데이터 파일 저장하기
아래 링크에서 labeeld_data.zip 파일을 다운받아 적절한 위치에 저장한 후 압축을 풀어주세요.
  
AI Hub의 '낚시성 기사 탐지 데이터'의 라벨링 데이터의 저장되어 있는 구조를 약간 변형하고 서브 디렉토리들의 압축을 풀어 저장해놓은 버전입니다.

드라이브 링크: 
https://drive.google.com/drive/folders/1kC1-GzNGQY-kX5rv0AH1JAtXz4NkyamD?usp=sharing

### 데이터 설명

labeled_data 디렉토리 안에는 4개의 디렉토리가 있습니다.

##### training_set
전체 labeled training data를 담은 디렉토리입니다.
##### training_set_mini
labeled training data 중 2개의 sub directory만 담은 작은 크기의 데이터입니다.
##### validation_set
전체 labeled validation data를 담은 디렉토리입니다.
##### training_set_mini
labeled validation data 중 1개의 sub directory만 담은 작은 크기의 데이터입니다.


In [ ]:
# github에서 미니 데이터를 불러오는 부분
#!git clone https://github.com/hyunmin5366/NLPClassification.git

Cloning into 'NLPClassification'...
remote: Enumerating objects: 49282, done.
remote: Total 49282 (delta 0), reused 0 (delta 0), pack-reused 49282
Receiving objects: 100% (49282/49282), 79.63 MiB | 10.48 MiB/s, done.
Resolving deltas: 100% (24/24), done.
Updating files: 100% (49274/49274), done.


----
### 데이터 불러오기
'load_entire_labeled_data' 함수를 통해 데이터를 불러옵니다.

##### 첫번째 argument
  
제가 보내드린 데이터 디렉토리의 경로
  
(ex. 'training_set_mini', 'training_set', 'validation_set_mini'. 'valication_set'등의 디렉토리)

##### 두번째 argument
서브 디렉토리(ex. TL_Part1_Clickbait_Auto_EC 등)의 압축을 풀고 로드할 것인지 아닌지를 결정하는 인자.
  
1: 압축을 풂, 0: 압축을 풀지 않음
  
제가 미리 압축을 다 풀어놓은 상태이므로, 0으로 놓고 실행해주세요.


In [ ]:
# 파일의 경로를 적절하게 변형해주세요.
# ex) train_df = data_load.load_entire_labeled_data('./labeled_data/training_set', 0)
train_df = data_load.load_entire_labeled_data('/content/NLPClassification/training_set_mini', 0)
test_df = data_load.load_entire_labeled_data('/content/NLPClassification/validation_set_mini', 0)

[INFO] loading labeled data...
[INFO] program was done.
[INFO] execution time:  189.6964361667633
[INFO] loading labeled data...
[INFO] program was done.
[INFO] execution time:  5.878288269042969


In [ ]:
def prep(train_df):
  #수치화
  train_df['processLevel'] = train_df['processLevel'].map({'하': 0, '중': 1, '상':2})
  train_df['processType'] = train_df['processType'].map({'A': 0, 'D': 1})
  train_df['partNum'] = train_df['partNum'].map({'P1':0, 'P2': 1})

  #Newtitle, ReferSentenceInfo 제거
  train_df = train_df.drop('newTitle', axis=1)
  train_df = train_df.drop('referSentenceInfo', axis=1)

  #Title, Subtitle, News Content 병합
  train_df['newsTitle'] += ': ' + train_df['newsSubTitle']
  train_df = train_df.drop('newsSubTitle', axis=1)
  train_df['newsText'] = train_df[['newsTitle', 'newsContent']].agg(''.join, axis=1)
  return train_df

In [ ]:
# 2. Preprocessing
train_df2 = prep(train_df)
test_df2 = prep(test_df)

In [ ]:
class TestCBDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 13]
        label = self.df.iloc[idx, 12]
        return text, label

In [ ]:
CB_train_dataset = TestCBDataset(train_df2)
train_loader = DataLoader(CB_train_dataset, batch_size=8, shuffle=True)

In [ ]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-5)

itr = 1
p_itr = 100
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):

    for text, label in train_loader:
        optimizer.zero_grad()

        encoded = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        encoded, label = encoded.to(device), label.to(device)
        outputs = model(**encoded, labels=label)

        loss = outputs.loss
        logits = outputs.logits

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(label)
        total_correct += correct.sum().item()
        total_len += len(label)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

In [ ]:
model.eval()

CB_eval_dataset = TestCBDataset(test_df2)
eval_loader = DataLoader(CB_eval_dataset, batch_size=8, shuffle=False)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:

    encoded = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    encoded, label = encoded.to(device), label.to(device)
    outputs = model(**encoded, labels=label)

    logits = outputs.logits

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(label)
    total_correct += correct.sum().item()
    total_len += len(label)

print('Test accuracy: ', total_correct / total_len)


<ipython-input-11-9b6dbf7e1ed1>:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)


Test accuracy:  0.8649249583101724
